In [1]:
OUTPUT_PATH = '../../data/'

In [2]:
%run '../../schema_definition/schema.py'

In [3]:
import sys
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [4]:
from pyspark.shell import spark
from pyspark.sql import functions as F

21/10/16 04:14:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.8.9 (default, Aug 21 2021 15:53:23)
Spark context Web UI available at http://localhost:4042
Spark context available as 'sc' (master = local[*], app id = local-1634318074587).
SparkSession available as 'spark'.


- When the number of events reach 1000, output the events to a JSON file.
- The output filenames should have a batch number e.g. the second 1000 records will go into a file called 2.json and so on.

In [8]:
gold_df = spark.read.parquet(OUTPUT_PATH + "gold")
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT *
                        FROM fastest_response
                        """
fastest_response_df = spark.sql(fastest_response_query)
# fastest_response_df.coalesce(1).write.csv(OUTPUT_PATH + 'response', sep=',', header=True, mode='overwrite')
fastest_response_df.coalesce(1).write.option("maxRecordsPerFile", 1000).json(OUTPUT_PATH + 'response', mode='overwrite')
df = fastest_response_df.toPandas()
df

,AccountID,CODE,ImplementedDate,ActiveIndicator,AccountType,Service,BU,RequestDate,AccountStatus,StatusCode,Amount,Version,AgentID,Fibre,LastUpdatedDate,PropertyType,PostCode,HashKey,date,Response
0,15368,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-17,PAID,8,3.94,v1_0,307508,EN121794032-N-5,2017-09-07 11:09:00,RESI,2587,ae52557431727f096d968da7265c14abeb3bb3afa0fb4a...,2021-10-16,-21
1,23232,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-10,PAID,7,126.96,v1_0,307508,EN121542407-N-12,2017-09-07 11:04:00,RESI,2587,e4aad9f5903db9c97d5991d49f50e986b086e4c5343ca9...,2021-10-16,-28
2,232323,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-09,PAID,6,15.43,v1_0,307508,EN121491293-N-9,2017-09-07 11:05:00,RESI,2587,6e463c062b46ce88656b7971c482bb5489b9e012214848...,2021-10-16,-29
3,307263,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-09,PAID,6,3.49,v1_0,307508,EN121491293-N-13,2017-09-07 11:05:00,RESI,2587,6e2f04ff85422285e6cfe08508144b47262c77a590dee3...,2021-10-16,-29
4,415740,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,8,3.31,v1_0,307312,EA983_54004586547-N-11,2017-09-11 10:44:00,RESI,2581,9057e3e8fc73f87b752b99e782400367c2ec2390358e72...,2021-10-16,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29636,10078,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-28,PAID,7,6.92,v1_0,307510,EN122165150-N-15,2017-09-07 12:44:00,RESI,2587,e6a92b6e6004c91e49608b9e9910abbb7644c3294acd6a...,2021-10-16,-10
29637,10095,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,1,0.46,v1_0,307510,EN122228188-N-5,2017-09-07 12:48:00,RESI,2587,a620f509f0657fd9a9e10f2b1db5645dd3bd518f1bcb2a...,2021-10-16,-9
29638,10036,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,7,8.53,v1_0,307510,EN122231542-N-9,2017-09-07 12:48:00,RESI,2587,39ba2d4d960ac2baf636a187ad2e18eea7470aa5f1d94e...,2021-10-16,-9
29639,10053,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-29,PAID,8,7.44,v1_0,307510,EN122225528-N-9,2017-09-07 12:48:00,RESI,2587,7a531111b3b8f0efe538d47cf521d2fd95475824e623da...,2021-10-16,-9


In [19]:
import plotly.express as px

fig = px.scatter(df, x='Response', y='CODE', title='Fastest response')
fig.show()